In [ ]:
import ast
import collections
from collections import Counter
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import sqlite3

pd.set_option('display.width', 1000)
sb.set_palette(sb.color_palette('Paired'))

NAR = '20'
print("NAR:",NAR)

In [ ]:
class SQLiteContext:
    def __init__(self):
        self.conn = sqlite3.connect('/home/ghsong/django-substring/mysite/db.sqlite3')
        self.conn.row_factory = dict_factory
    
    def __enter__(self):
        return self.conn.cursor()
    
    def __exit__(self, type, value, traceback):
        self.conn.commit()
        self.conn.close()

In [ ]:
def execute_query(query):
    with SQLiteContext() as cur:
        cur.execute(query)
        return cur.fetchall()

In [ ]:
query_tmpl = 'SELECT exp_date, algorithm_id, dataset_id, parameter, result FROM projectManager_expitem WHERE project_id=1 AND invalid=0 {} ORDER BY -exp_date'

In [ ]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [ ]:
def str2dict(s):
    return ast.literal_eval(s)

In [ ]:
alg_list = ['PrefixSearch_6.32', 'PkwiseSynSearch_2.00', 'FaerieSynSearch_1.02']
alg_name_final = 'PrefixSearch_6.32'
# alg_list = ['PrefixSearch_6.20']

In [ ]:
# setting_list = ['RSS-NAIVE', 'RSS-LPR', 'RSS-C', 'RSS-CP', 'RSS-CPL', 'RSS-CPLPR', 'RSS-PKWISE']#, 'RSS-PKWISE2', 'RSS-PKWISE3']
# setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L', 'RSS-PR', 'RSS-CP', 'RSS-CL', 'RSS-PL','RSS-CPL', 'RSS-CPLPR']
# setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L',  'RSS-CP', 'RSS-CL', 'RSS-PL','RSS-CPL', 'RSS-CPPR', 'RSS-CPLPR']
# setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-CP', 'RSS-CL', 'RSS-PL', 'RSS-CPL']
setting_list = ['RSS-NAIVE', 'RSS-CL', 'RSS-PL', 'RSS-CP', 'RSS-CPL', 'PKDUCK-W', 'PKWISE-R', 'FAERIE-R']

def get_setting_label(aid, dict_param):
#     print(aid, dict_param)
    alg_name = dict_aid[aid]
#     assert alg_name in alg_list
    if alg_name.startswith('PrefixSearch') or alg_name.startswith('ZeroPrefixSearch'):
        return get_setting_label_PrefixSearch(dict_param)
    elif alg_name.startswith('PkwiseSynSearch'):
        return get_setting_label_PkwiseSynSearch(dict_param)
    elif alg_name.startswith('FaerieSynSearch_'):
        return 'FAERIE-R'
    elif alg_name.startswith('NaiveContainmentSearch_'):
        return 'RSS-JC-NAIVE'
    elif alg_name.startswith('ContainmentPrefixSearch_'):
        return 'RSS-JC'

    
def get_setting_label_PrefixSearch(dict_param):
    bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
    idx_impl = dict_param['index_impl']
    if not bLF and not bPF and idx_impl == 'None': return 'RSS-NAIVE'
    elif not bLF and not bPF and idx_impl == 'Naive': return None#return 'RSS-INDEX'
    elif not bLF and not bPF and idx_impl == 'Count': return 'RSS-C'
    elif not bLF and not bPF and idx_impl == 'Position': return 'RSS-P'
    elif bLF and not bPF and idx_impl == 'None': return 'RSS-L'
    elif not bLF and bPF and idx_impl == 'None': return 'RSS-R'
    
    elif not bLF and not bPF and idx_impl == 'CountPosition': return 'RSS-CP'
    elif bLF and not bPF and idx_impl == 'Count': return 'RSS-CL'
    elif bLF and not bPF and idx_impl == 'Position': return 'RSS-PL'
    elif not bLF and bPF and idx_impl == 'Count': return 'RSS-CR'
    elif not bLF and bPF and idx_impl == 'Position': return 'RSS-PR'
    elif bLF and bPF and idx_impl == 'None': return 'PKDUCK-W'
    
    elif bLF and not bPF and idx_impl == 'CountPosition': return 'RSS-CPL'
    elif not bLF and bPF and idx_impl == 'CountPosition': return 'RSS-CPR'
    elif bLF and bPF and idx_impl == 'Count': return 'RSS-CLR'
    elif bLF and bPF and idx_impl == 'Position': return 'RSS-PLR'
    
    elif bLF and bPF and idx_impl == 'CountPosition': return 'RSS-CPLR'
    
    else: return None

def get_setting_label_PkwiseSynSearch(dict_param):
    qlen = dict_param['qlen']
    kmax = dict_param['kmax']
#     if kmax == '1': return 'RSS-PKWISE-k1'
#     elif kmax == '2': return 'RSS-PKWISE-k2'
    if kmax == 'opt': return 'PKWISE-R'
#     if   qlen == '1' and kmax == '1': return 'RSS-PKWISE'
#     elif qlen == '3' and kmax == '2': return 'RSS-PKWISE'
#     elif qlen == '5' and kmax == '2': return 'RSS-PKWISE'
#     elif qlen == '7' and kmax == '2': return 'RSS-PKWISE'
#     elif qlen == '9' and kmax == '2': return 'RSS-PKWISE'
    else: return None

In [ ]:
dict_alg = {}
dict_aid = {}
with SQLiteContext() as cur:
    cur.execute("SELECT id, name, version FROM projectManager_algorithm WHERE project_id=1")
    rows = cur.fetchall()
    for row in rows: 
        dict_alg[row['name']+'_'+row['version']] = row['id']
        dict_aid[row['id']] = row['name']+'_'+row['version']

In [ ]:
DataInfo = collections.namedtuple('DataInfo', ['name', 'size', 'nr', 'qlen', 'lr', 'nar'])

In [ ]:
dict_data = {}
dict_did = {}
with SQLiteContext() as cur:
    cur.execute("SELECT id, name FROM projectManager_dataset WHERE project_id=1 and id >= 498")
    rows = cur.fetchall()
    for row in rows:
        tokens = row['name'].rsplit('_')
        if len(tokens) == 5: 
            dname, size, nr, qlen, lr = tokens
            nar = 'a-1'
        elif len(tokens) == 6:
            dname, size, nr, qlen, lr, nar = tokens
        size, nr, qlen, lr, nar = size[1:], nr[1:], qlen[1:], lr[1:], nar[1:]
        try: dict_data[dname]
        except: dict_data[dname] = {}
        dict_data[dname][(size, nr, qlen, lr, nar)] = row['id']
        dict_did[row['id']] = (dname, size, nr, qlen, lr, nar)

In [ ]:
def get_did_list(data_info):
    assert type(data_info) == DataInfo
    dname, size, nr, qlen, lr, nar = data_info
    const_idx_list = list(map(lambda x:x[0], filter(lambda x: x[1]!='*', enumerate([size, nr, qlen, lr, nar]))))
    assert len(const_idx_list) == 4
    target_idx = [size, nr, qlen, lr, nar].index('*')
    data_item_list = dict_data[dname].items()
    for idx in const_idx_list:
        data_item_list = list(filter(lambda x: x[0][idx] == data_info[idx+1], data_item_list))
    return list(sorted(map(lambda x: (x[0][target_idx], x[1]), data_item_list), key=lambda x:float(x[0])))

In [ ]:
def get_size_did_list(dname, nr, qlen, lr, nar):
    return list(
        sorted(
            map(lambda x: (x[0][0], x[1]), 
                filter(lambda x: x[0][1]==nr and x[0][2]==qlen and x[0][3]==lr, dict_data[dname].items()))
        , key=lambda x: int(x[0])))

In [ ]:
def get_qlen_did_list(dname, size, nr, lr, nar):
    return list(
        sorted(
            map(lambda x: (x[0][2], x[1]), 
                filter(lambda x: x[0][0]==size and x[0][1]==nr and x[0][3]==lr, dict_data[dname].items()))
        , key=lambda x: int(x[0])))

In [ ]:
# marker_list = itertools.cycle(['+', 'x', 'o', '^', 'D', 's', 'v'])
marker_list = ['+', 'x', 'o', '^', 'D', 's', 'v']
dashes_list = itertools.cycle([0])

In [ ]:
def parse_data_info(data_info):
    assert type(data_info) == DataInfo
    dname, size, nr, qlen, lr, nar = data_info
    return dname, size, nr, qlen, lr, nar

In [ ]:
def parse_query_row(row, more_features=None):
    aid = int(row['algorithm_id'])
    dict_param = str2dict(row['parameter'])
    dict_rslt = str2dict(row['result'])
    dict_param['qlen'] = dict_rslt['Dataset_qlen']

    output = {}
    output['id'] = dict_rslt['Alg_ID']
    output['nq'] = int(dict_rslt['Dataset_numSearched'])
    output['n'] = int(dict_rslt['Dataset_numIndexed'])
    try: output['n_doc'] = int(dict_rslt['Dataset_numDoc'])
    except: pass
    output['nr'] = int(dict_rslt['Dataset_nr'])
    output['qlen'] = int(dict_rslt['Dataset_qlen'])
    output['lr'] = float(dict_rslt['Dataset_lr'])
    try: output['nar'] = float(dict_rslt['Dataset_nar'])
    except: output['nar'] = None
    output['Dataset_name'] = dict_rslt['Dataset_Name'].split('_')[0]
    output['theta'] = float(dict_param['theta'])
    output['setting'] = get_setting_label(aid, dict_param)
#     if output['setting'] not in setting_list: return None
    output['alg'] = dict_aid[aid]
    output['label'] = output['alg'] +'_' + output['setting']
    output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
    output['Time_QS_Total'] = float(dict_rslt['Time_QS_Total'])/1000
    output['Time_TS_Total'] = float(dict_rslt['Time_TS_Total'])/1000
    output['Time_Search'] = output['Time_QS_Total'] + output['Time_TS_Total']
    output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
    output['Time_IndexFilter'] = (float(dict_rslt['Time_QS_IndexFilter']) + float(dict_rslt['Time_TS_IndexFilter']))/1000
    output['Time_Validation'] = (float(dict_rslt['Time_QS_Validation']) + float(dict_rslt['Time_TS_Validation']))/1000
    output['Time_BuildIndex'] = float(dict_rslt['Time_BuildIndex'])/1000
    output['Num_Result'] = int(dict_rslt['Num_Result'])
    output['Num_QS_Verified'] = int(dict_rslt['Num_QS_Verified'])
    output['Num_TS_Verified'] = int(dict_rslt['Num_TS_Verified'])
    output['Num_Verified'] = output['Num_QS_Verified'] + output['Num_TS_Verified']
    output['Len_Retrieved'] = int(dict_rslt['Len_QS_Retrieved']) + int(dict_rslt['Len_TS_Retrieved'])
    output['Space_Index'] = int(dict_rslt['Space_Index'])/1e6 if dict_rslt['Space_Index'] != 'null' else 0
    
    assert more_features is None or type(more_features) == list or more_features == '*'
    if more_features is not None:
        if more_features == '*': more_features = list(sorted(dict_rslt.keys() - output.keys()))
        for feat in more_features:
            val = dict_rslt[feat]
            try: output[feat] = int(val)
            except:
                try: output[feat] = float(val)/1000
                except: output[feat] = val
    return output

In [ ]:
def df_time_by_filtering(data_info, alg_name=alg_name_final):
    dname, size, nr, qlen, lr, nar = parse_data_info(data_info)
    did = dict_data[dname][(size, nr, qlen, lr, nar)]
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
    query = query_tmpl.format(predicates)
    output_list = []
    for row in execute_query(query):
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])
        output = {}
        output['algorithm_id'] = row['algorithm_id']
        output['theta'] = float(dict_param['theta'])
#         bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
#         idx_impl = dict_param['index_impl']
        output['setting'] = get_setting_label(output['algorithm_id'], dict_param)
        if output['setting'] is None: continue
        output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
        output['Time_SearchPerQuery_MEAN'] = float(dict_rslt['Time_SearchPerQuery_MEAN'])/1000
        output['Num_QS_Verified'] = int(dict_rslt['Num_QS_Verified'])
        output['Num_TS_Verified'] = int(dict_rslt['Num_TS_Verified'])
        output['Num_Verified'] = int(dict_rslt['Num_QS_Verified']) + int(dict_rslt['Num_TS_Verified'])
        output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_size(data_info, alg_list=alg_list):
    size_did_list = get_did_list(data_info)
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    output_list = []
    for size, did in size_did_list:
        predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            output = parse_query_row(row)
            if output is None: continue
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
#     df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_qlen(data_info, alg_list=alg_list):
    qlen_did_list = get_did_list(data_info)
    qlen_list = list(map(lambda x:int(x[0]), qlen_did_list))
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    output_list = []
    for qlen, did in qlen_did_list:
        predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            output = parse_query_row(row)
            if output is None: continue
            output['qlen'] = int(qlen)
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
#     df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_nr(data_info, alg_list=alg_list):
    nr_did_list = get_did_list(data_info)
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    output_list = []
    for nr, did in nr_did_list:
        predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            output = parse_query_row(row)
            if output is None: continue
            output['nr'] = int(nr)
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
#     df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_time_by_lr(data_info, alg_list=alg_list):
    lr_did_list = get_did_list(data_info)
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    output_list = []
    for lr, did in lr_did_list:
        predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
        query = query_tmpl.format(predicates)
        for row in execute_query(query):
            output = parse_query_row(row)
            if output is None: continue
            output['lr'] = float(lr)
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
#     df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_acc(alg_name=alg_name_final):
    output_list = []
    aid = dict_alg[alg_name]
    predicates = ' '.join(['AND algorithm_id={}'.format(aid)])
    query = query_tmpl.format(predicates)
    for row in execute_query(query):
        aid = int(row['algorithm_id'])
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])
        bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
        idx_impl = dict_param['index_impl']

        output = {}
        output['data_name'] = dict_rslt['Dataset_Name'].split('_',1)[0]
        output['n'] = int(dict_rslt['Dataset_nt'])
        output['nr'] = int(dict_rslt['Dataset_nr'])
        output['qlen'] = int(dict_rslt['Dataset_qlen'])
        output['theta'] = float(dict_param['theta'])
        output['setting'] = get_setting_label(aid, dict_param)
        if output['setting'] is None: continue
        output['Num_Result'] = int(dict_rslt['Num_Result'])
        output['Num_QS_Result'] = int(dict_rslt['Num_QS_Result'])
        output['Num_TS_Result'] = int(dict_rslt['Num_TS_Result'])
        output_list.append(output)
        
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    return df

In [ ]:
def df_from_PrefixSearchFilterPowerTest(setting_list, path='./ZeroPrefixSearch.out'):
#     if setting_list is None: setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L', 'RSS-CP', 'RSS-CL', 'RSS-PL','RSS-CPL', 'RSS-CPR', 'RSS-CPLR']
    output_list = []
    with open(path) as f:
        for line in f:
            token_list = line.strip().split('\t')
            row = {k:v for k,v in map(lambda x:x.split(':',1), token_list)}
            dict_param = {k:v for k,v in map(lambda x:x.split(':',1), row['Param'][1:-3].split(', '))}
            
            output = {}
            output['theta'] = float(dict_param['theta'])
            output['data_name'] = row['Dataset_Name'].split('_',1)[0]
            output['n'] = int(row['Dataset_nt'])
            output['nr'] = int(row['Dataset_nr'])
            output['qlen'] = int(row['Dataset_qlen'])
            output['setting'] = get_setting_label(56, dict_param)
            output['Num_QS_Verified'] = int(row['Num_QS_Verified'])
            output['Num_TS_Verified'] = int(row['Num_TS_Verified'])
            output['Num_Verified'] = int(row['Num_QS_Verified']) + int(row['Num_TS_Verified'])
            output['Len_Verified'] = int(row['Len_QS_Verified']) + int(row['Len_TS_Verified'])
            output['Time_QS_Total'] = float(row['Time_QS_Total'])/1000
            output['Time_TS_Total'] = float(row['Time_TS_Total'])/1000
            output['Time_Total'] = float(row['Time_Total'])/1000
            output['Time_SearchPerQuery_MEAN'] = float(row['Time_SearchPerQuery_MEAN'])/1000
            try: output['Time_TS_searchRecordPF.pkduck'] = float(row['Time_TS_searchRecordPF.pkduck'])/1000
            except: output['Time_TS_searchRecordPF.pkduck'] = 0
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.setting = pd.Categorical(df.setting, categories=setting_list)
    dfg = df.groupby(by=['data_name', 'n', 'nr', 'qlen', 'setting', 'theta']).mean().reset_index()
    return dfg

In [ ]:
setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L', 'RSS-R', 'RSS-CL', 'RSS-PL', 'RSS-CP', 'RSS-CR', 'RSS-PR', 'PKDUCK-W', 'RSS-CPL', 'RSS-CPR', 'RSS-CLR', 'RSS-PLR', 'RSS-CPLR']
path = 'ZeroPrefixSearch.out.repeat10'
output_list = []
with open(path) as f:
    for idx, line in enumerate(f):
        token_list = line.strip().split('\t')
        row = {k:v for k,v in map(lambda x:x.split(':',1), token_list)}
        dict_param = {k:v for k,v in map(lambda x:x.split(':',1), row['Param'][1:-3].split(', '))}

        output = {}
        output['theta'] = float(dict_param['theta'])
        output['data_name'] = row['Dataset_Name'].split('_',1)[0]
        output['n'] = int(row['Dataset_nt'])
        output['nr'] = int(row['Dataset_nr'])
        output['qlen'] = int(row['Dataset_qlen'])
        output['setting'] = get_setting_label(56, dict_param)
        output['Num_QS_Verified'] = int(row['Num_QS_Verified'])
        output['Num_TS_Verified'] = int(row['Num_TS_Verified'])
        output['Num_Verified'] = int(row['Num_QS_Verified']) + int(row['Num_TS_Verified'])
        output['Len_Verified'] = int(row['Len_QS_Verified']) + int(row['Len_TS_Verified'])
        output['Time_Total'] = float(row['Time_Total'])/1000
        output['Time_SearchPerQuery_MEAN'] = float(row['Time_SearchPerQuery_MEAN'])/1000
        try: output['Time_TS_searchRecordPF.pkduck'] = float(row['Time_TS_searchRecordPF.pkduck'])/1000
        except: output['Time_TS_searchRecordPF.pkduck'] = 0
        output_list.append(output)
#         if idx > 0: break
df = pd.DataFrame.from_dict(output_list)
df.setting = pd.Categorical(df.setting, categories=setting_list)
dfg = df.groupby(by=['data_name', 'n', 'qlen', 'setting', 'theta']).mean().reset_index()
dfg

In [ ]:
def df_from_ExactPrefixSearch():
    path = './ExactPrefixSearch.txt'
    output_list = []
    with open(path) as f:
        for line in f:
            token_list = line.strip().split('\t')
            row = {k:v for k,v in map(lambda x:x.split(':',1), token_list)}
            dict_param = {k:v for k,v in map(lambda x:x.split(':',1), row['Param'][1:-3].split(', '))}
            output = {}
            output['theta'] = float(dict_param['theta'])
            output['data_name'] = row['Dataset_Name'].split('_',1)[0]
            output['n'] = int(row['Dataset_nt'])
            output['nr'] = int(row['Dataset_nr'])
            output['qlen'] = int(row['Dataset_qlen'])
            output['Num_QS_Result'] = int(row['Num_QS_Result'])
            output['Num_TS_Result'] = int(row['Num_TS_Result'])
            output['Num_Result'] = int(row['Num_Result'])
            output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    df.drop_duplicates(inplace=True)
    return df

In [ ]:
def df_index_stat(data_info, alg_list=alg_list):
    dname, size, nr, qlen, lr = parse_data_info(data_info)
    did = dict_data[dname][(size, nr, qlen, lr)]
    predicates_alg = ' OR '.join(['algorithm_id={}'.format(dict_alg[alg_name]) for alg_name in alg_list])
    predicates = ' AND '.join(['', 'dataset_id={}'.format(did), '({})'.format(predicates_alg)])
    query = query_tmpl.format(predicates)
    output_list = []
    for row in execute_query(query):
        aid = int(row['algorithm_id'])
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])

        output = {}
        output['data_name'] = dict_rslt['Dataset_Name'].split('_',1)[0]
        output['n'] = int(dict_rslt['Dataset_nt'])
        output['nr'] = int(dict_rslt['Dataset_nr'])
        output['qlen'] = int(dict_rslt['Dataset_qlen'])
        output['theta'] = float(dict_param['theta'])
        output['setting'] = get_setting_label(aid, dict_param)
        if output['setting'] is None: continue
        output['Time_BuildIndex'] = float(dict_rslt['Time_BuildIndex'])/1000
        output['Space_Index'] = 0 if dict_rslt['Space_Index'] == 'null' else int(dict_rslt['Space_Index'])
        output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
    return df

In [ ]:
def plot_verify_by_filtering(theta, qlen):
    df = df_from_PrefixSearchFilterPowerTest()
    df = df[df.theta == float(theta)][df.qlen == int(qlen)]
    f, axes = plt.subplots(1, 3, figsize=(21, 6), sharex=True, sharey=False)
    sb.catplot(x='setting', y='Num_Verified', hue='setting', data=df[df.data_name == 'WIKI'], kind='bar', ax=axes[0])
    sb.catplot(x='setting', y='Num_Verified', hue='setting', data=df[df.data_name == 'PUBMED'], kind='bar', ax=axes[1])
    sb.catplot(x='setting', y='Num_Verified', hue='setting', data=df[df.data_name == 'AMAZON'], kind='bar', ax=axes[2])
    axes[0].set(ylabel='# Verified', yscale='log')
    axes[1].set(ylabel='# Verified', yscale='log')
    axes[2].set(ylabel='# Verified', yscale='log')
    for i in range(2,5): plt.close(i)

In [ ]:
def plot_time_by_filtering(data_info, alg_name=alg_name_final):
    df = df_time_by_filtering(data_info, alg_name=alg_name_final)
    f, axes = plt.subplots(1, 3, figsize=(21, 6), sharex=False, sharey=False)
    sb.catplot(x='theta', y='Time_Total', hue='setting', data=df, kind='bar', ax=axes[0])
    sb.catplot(x='theta', y='Time_SearchPerQuery_MEAN', hue='setting', data=df, kind='bar', ax=axes[1])
    sb.catplot(x='theta', y='Num_Verified', hue='setting', data=df, kind='bar', ax=axes[2])
    axes[0].set(ylabel='Total Time (sec)', yscale='log')
    axes[1].set(ylabel='Mean query time (s)', yscale='log')
    axes[2].set(ylabel='# Verified', yscale='log')
    for i in range(2,5): plt.close(i)

In [ ]:
# def get_hue_style_orders(df):
#     counter = {'setting':Counter(), 'alg':Counter()}
#     for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
#         for val in set(df[df.theta==theta]['setting']): counter['setting'][val] += 1
#         for val in set(df[df.theta==theta]['alg']): counter['alg'][val] += 1
#     hue_order = list(sorted(counter['setting'].keys(), key=counter['setting'].get, reverse=True))
#     style_order = list(sorted(counter['alg'].keys(), key=counter['alg'].get, reverse=True))
#     return hue_order, style_order

In [ ]:
def plot_time_by_size(data_info, alg_list=alg_list, setting_list=setting_list, y='Time_SearchPerQuery_MEAN', ylabel='Mean query time (s)', theta_list=[0.6, 0.7, 0.8, 0.9, 1.0], savefig=False):
    df = df_time_by_size(data_info, alg_list=alg_list)
    f, axes = plt.subplots(1, len(theta_list), figsize=(5*len(theta_list), 5), sharex=False, sharey=False, squeeze=False)
    f.suptitle(data_info)
    x = "n_doc" if data_info.name.endswith('-DOC') else "n"
    for i, theta in enumerate(theta_list):
        ax = axes[0][i]
        sb.lineplot(data=df[df.theta==theta], x=x, y=y, hue="setting", hue_order=setting_list, style="alg", style_order=alg_list, markers=True, alpha=0.3, ax=ax)
        ax.set(ylabel=ylabel, xscale='log', yscale='log', title='theta=%.1f'%theta)
        if i == len(theta_list)-1: ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: ax.get_legend().remove()
    if savefig: plt.savefig(f'plot_time_by_size_{data_info.name}.png', dpi=400, bbox_inches='tight')

In [ ]:
def plot_time_by_qlen(data_info, alg_list=alg_list, y='Time_SearchPerQuery_MEAN', ylabel='Mean query time (s)', savefig=False):
    dname, size, nr, qlen, lr, nar = parse_data_info(data_info)
    qlen_list = list(map(lambda x:int(x[0]), get_did_list(data_info)))
    df = df_time_by_qlen(data_info, alg_list=alg_list)
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="qlen", y=y, hue="setting", hue_order=setting_list, style="alg", style_order=alg_list, markers=True, ax=axes[i])
        axes[i].set(xticks=qlen_list, ylabel=ylabel, yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()
    if savefig: plt.savefig(f'plot_time_by_qlen_{data_info.name}.png', dpi=400)

In [ ]:
def plot_time_by_nr(data_info, alg_list=alg_list, y='Time_SearchPerQuery_MEAN', ylabel='Mean query time (s)', savefig=False):
    df = df_time_by_nr(data_info, alg_list=alg_list)
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="nr", y=y, hue="setting", hue_order=setting_list, style="alg", style_order=alg_list, markers=True, ax=axes[i])
        axes[i].set(ylabel=ylabel, xscale='log', yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()
    if savefig: plt.savefig(f'plot_time_by_nr_{data_info.name}.png', dpi=400)

In [ ]:
def plot_time_by_lr(data_info, alg_list=alg_list, y='Time_SearchPerQuery_MEAN', ylabel='Mean query time (s)'):
    df = df_time_by_lr(data_info, alg_list=alg_list)
    lr_list = list(map(lambda x:float(x[0]), get_did_list(data_info)))
    f, axes = plt.subplots(1, 5, figsize=(25, 5), sharex=False, sharey=False)
    for i, theta in zip(range(5), [0.6, 0.7, 0.8, 0.9, 1.0]):
        sb.lineplot(data=df[df.theta==theta], x="lr", y=y, hue="setting", hue_order=setting_list, style="alg", style_order=alg_list, markers=True, ax=axes[i])
        axes[i].set(xticks=lr_list, ylabel=ylabel, yscale='log', title='theta=%.1f'%theta)
        if i == 4: axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
        else: axes[i].get_legend().remove()

In [ ]:
def compare_alg(alg0_name, alg1_name):
    aid0 = dict_alg[alg0_name]
    aid1 = dict_alg[alg1_name]
    predicates = ' '.join(['AND (algorithm_id={} OR algorithm_id={})'.format(aid0, aid1)])
    query = query_tmpl.format(predicates)
    output_list = []
    for row in execute_query(query):
        dict_param = str2dict(row['parameter'])
        dict_rslt = str2dict(row['result'])
        bLF, bPF = map(lambda x: x == 'true', [dict_param[key] for key in ['bLF', 'bPF']])
        idx_impl = dict_param['index_impl']

        output = {}
        output['dataset'] = dict_rslt['Dataset_Name']
        output['size'] = dict_rslt['Dataset_Name']
        output['alg'] = dict_rslt['Alg_Name']+'_'+dict_rslt['Alg_Version']
        output['theta'] = float(dict_param['theta'])
        output['setting'] = get_setting_label(aid0, dict_param)
        if output['setting'] is None: continue
        output['Time_Total'] = float(dict_rslt['Time_Total'])/1000
        try: output['Time_QS_Total'] = float(dict_rslt['Time_QS_Total'])/1000
        except: output['Time_QS_Total'] = float(dict_rslt['Time_QS_Total'])/1000
        try: output['Time_TS_Total'] = float(dict_rslt['Time_TS_Total'])/1000
        except: output['Time_TS_Total'] = float(dict_rslt['Time_TS_Total'])/1000
        output['Time_Search'] = output['Time_QS_Total'] + output['Time_TS_Total']
        output['Num_Verified'] = int(dict_rslt['Num_QS_Verified']) + int(dict_rslt['Num_TS_Verified'])
        output['Num_QS_Result'] = int(dict_rslt['Num_QS_Result'])
        output['Num_TS_Result'] = int(dict_rslt['Num_TS_Result'])
        
        output_list.append(output)
    df = pd.DataFrame.from_dict(output_list)
#     df.setting = pd.Categorical(df.setting, categories=setting_list)
    
    df0 = df[df.alg==alg0_name].drop(['alg'], axis=1)
    df1 = df[df.alg==alg1_name].drop(['alg'], axis=1)
    df_merged = pd.merge(df0, df1, on=['dataset', 'size', 'theta', 'setting'], how='inner', suffixes=['_0','_1'])
    df_merged['diff_Time'] = df_merged['Time_Total_1'] - df_merged['Time_Total_0']
#     df_merged['diff_Time_ratio'] = (df_merged['Time_Total_1'] - df_merged['Time_Total_0'])/df_merged['Time_Total_0']
    df_merged['diff_Time_Search'] = df_merged['Time_Search_1'] - df_merged['Time_Search_0']
#     df_merged['diff_Time_Search_ratio'] = (df_merged['Time_Search_1'] - df_merged['Time_Search_0'])/df_merged['Time_Search_0']
    df_merged['diff_Verify'] = df_merged['Num_Verified_1'] - df_merged['Num_Verified_0']
#     df_merged['diff_Verify_ratio'] = (df_merged['Num_Verified_1'] - df_merged['Num_Verified_0'])/df_merged['Num_Verified_0']
    df_merged['diff_QS_Result'] = df_merged['Num_QS_Result_1'] - df_merged['Num_QS_Result_0']
    df_merged['diff_TS_Result'] = df_merged['Num_TS_Result_1'] - df_merged['Num_TS_Result_0']
    print('diff_Time:',df_merged['diff_Time'].mean())
#     print('diff_Time_ratio:',df_merged['diff_Time_ratio'].mean())
    print('diff_Time_Search:',df_merged['diff_Time_Search'].mean())
#     print('diff_Time_Search_ratio:',df_merged['diff_Time_Search_ratio'].mean())
    print('diff_Verify:',df_merged['diff_Verify'].mean())
#     print('diff_Verify_ratio:',df_merged['diff_Verify_ratio'].mean())
    print('diff_QS_Result:',df_merged['diff_QS_Result'].mean())
    print('diff_TS_Result:',df_merged['diff_TS_Result'].mean())
    df_merged = df_merged.reindex(sorted(df_merged.columns), axis=1)
    return df_merged

In [ ]:
# def performance_comparison():
    
#     def compare_algs(df, alg0, alg1, data_info=None, measure='Time_SearchPerQuery_MEAN'):
#         vals0 = df[df.setting==alg0][measure].values
#         vals1 = df[df.setting==alg1][measure].values
#         n = min(len(vals0), len(vals1))
#         print(measure, '' if data_info is None else data_info.name, '{}/{}'.format(alg0, alg1), vals0[:n]/vals1[:n])
    
#     df = df_from_PrefixSearchFilterPowerTest()
#     df = df[df.theta==theta0][df.n==n0][df.qlen==qlen0]
#     compare_algs(df, 'IF', 'NaivePF', measure='Num_Verified')
#     compare_algs(df, 'IF', 'ICF', measure='Num_Verified')
#     compare_algs(df, 'IF', 'IPF', measure='Num_Verified')
    
#     for data_info in [DataInfo('WIKI', '*', '107836', '3')]:#, DataInfo('PUBMED', '*', '79011', '3'), DataInfo('AMAZON', '*', '107836', '3')]:
#         df = df_time_by_size(data_info)
#         df = df[df.theta==theta0]
#         for setting in ['IPF', 'LF', 'PF']:
#             compare_algs(df, 'ICF', setting)
#         compare_algs(df, 'NaivePF', 'PF')
#         compare_algs(df, 'IF', 'NaivePF')

In [ ]:
def df_heuristic_accuracy():
    df0 = df_from_ExactPrefixSearch()
    df1 = df_acc()
    df = df0.merge(df1, on=['data_name', 'n', 'nr', 'qlen', 'theta'], suffixes=['0',''], how='inner')
    df = df[['data_name', 'n', 'nr', 'qlen', 'theta', 'setting', 'Num_Result0', 'Num_Result', 'Num_QS_Result0', 'Num_QS_Result', 'Num_TS_Result0', 'Num_TS_Result']]
    df['Acc'] = df['Num_Result']/df['Num_Result0']
    df['Acc_QS'] = df['Num_QS_Result']/df['Num_QS_Result0']
    df['Acc_TS'] = df['Num_TS_Result']/df['Num_TS_Result0']

    df = df[df.n==10000][df.nr==31622][df.setting=='RSS-CPL']
    return df